In [155]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
import sklearn.impute as imp
import sklearn.preprocessing as preproc
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import make_pipeline, Pipeline, make_union, FeatureUnion
from sklearn.compose import make_column_transformer, make_column_selector
import sklearn.tree as tree
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [230]:
df = pd.read_csv('raw.csv')
X, y = df.drop(labels='y', axis=1), df['y']

categorical_columns = make_column_selector(pattern='C_')
numerical_columns = make_column_selector(pattern='x_')
non_nan_categories = list(df.loc[:, df.columns.str.contains('C_')].dropna().apply(lambda x: x.unique()).to_numpy())

onehot_no_nans_encoder = preproc.OneHotEncoder(
    categories = non_nan_categories, 
    handle_unknown='ignore', 
    sparse_output=False
    )

numerical_imputer = imp.KNNImputer()

onehot_no_nans_transformer = make_column_transformer(
    (onehot_no_nans_encoder, categorical_columns),
    remainder="passthrough"
)

impute_x_transformer = make_column_transformer(
    (numerical_imputer, numerical_columns)
)

extract_categories_transformer = make_column_transformer(
    ("passthrough", categorical_columns)
)
preprocess_x = make_pipeline(onehot_no_nans_transformer, impute_x_transformer)


combine_category_and_imputed_x = make_union(preprocess_x, extract_categories_transformer)

imp.IterativeImputer()




combine_category_and_imputed_x.set_output(transform='pandas')
pipe = combine_category_and_imputed_x

new_data = pipe.fit_transform(X, y)
new_data.rename(mapper=lambda x: x.split('__')[-1], axis=1)

for i in range(1,6):
    y_label = f'C_{i}'
    X = new_data.drop


FeatureUnion(transformer_list=[('pipeline',
                                Pipeline(steps=[('columntransformer-1',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   transformers=[('onehotencoder',
                                                                                  OneHotEncoder(categories=[array(['H', 'K', 'I', 'G', 'J'], dtype=object),
                                                                                                            array(['H', 'I'], dtype=object),
                                                                                                            array(['K', 'G', 'I', 'J', 'H'], dtype=object),
                                                                                                            array(['J', 'I', 'G', 'H', 'K'], dtype=object),
                                                                                                            array(['J', 'G...
                                                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f7b345780a0>)])),
                                                ('columntransformer-2',
                                                 ColumnTransformer(transformers=[('knnimputer',
                                                                                  KNNImputer(),
                                                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f7b3457bdc0>)]))])),
                               ('columntransformer',
                                ColumnTransformer(transformers=[('passthrough',
                                                                 'passthrough',
                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f7b345780a0>)]))])

In [175]:
X[categorical_columns]
non_nan_categories

[array(['H', 'K', 'I', 'G', 'J'], dtype=object),
 array(['H', 'I'], dtype=object),
 array(['K', 'G', 'I', 'J', 'H'], dtype=object),
 array(['J', 'I', 'G', 'H', 'K'], dtype=object),
 array(['J', 'G', 'K', 'H', 'I'], dtype=object)]

In [142]:
dtree = tree.DecisionTreeRegressor(min_samples_leaf=.05)
dtree.fit(X, y)
fig, axes = plt.subplots(1, 1, figsize=(10,10), dpi=600)
tree.plot_tree(dtree, filled=True, feature_names=X.columns)

ValueError: could not convert string to float: 'I'

In [ ]:
sns.heatmap(new.corr(), cmap='BrBG', vmin=-1, vmax=1)

In [ ]:
non_nan_categories = list(df.loc[:, df.columns.str.contains('C')].dropna().apply(lambda x: x.unique()).to_numpy())
onehot_no_nans = preproc.OneHotEncoder(categories = cats, handle_unknown='ignore', sparse=False)